In [1]:
!python --version

Python 3.6.15


In [1]:
!python -m pip install torch==1.9.0

In [2]:
!pip install transformers

In [5]:
!pip install zero_shot_re

  Using cached zero_shot_re-0.0.4-py3-none-any.whl (6.6 kB)
  Using cached jupyterlab-2.2.9-py3-none-any.whl (7.9 MB)
     |████████████████████████████████| 14.5 MB 1.5 MB/s            
     |████████████████████████████████| 2.6 MB 492 kB/s            
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
     |████████████████████████████████| 9.9 MB 1.2 MB/s            
     |████████████████████████████████| 3.3 MB 1.0 MB/s            
  Using cached MarkupSafe-2.0.1-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (30 kB)
     |████████████████████████████████| 56 kB 2.3 MB/s            
     |████████████████████████████████| 178 kB 2.1 MB/s            
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
     |████████████████████████████████| 60 kB 1.5 MB/s            
     |████████████████████████████████| 552 kB 2.5 MB/s            
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
     |██████████████████████████████

In [3]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("fractalego/fewrel-zero-shot")

relations = ['noble title', 'founding date', 'occupation of a person']
extractor = RelationExtractor(model, tokenizer, relations)
ranked_rels = extractor.rank(text='John Smith received an OBE', head='John Smith', tail='OBE')
print(ranked_rels)

/home/gs/Info/IA/NLP/fewrel_zero_shot/venv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 1.34G/1.34G [05:45<00:00, 3.88MB/s]
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|█████████

[('noble title', 0.9003546684980392), ('occupation of a person', 0.0009881258010864258), ('founding date', 0.00013267993927001953)]


Le modèle n'est pas adapté pour distinguer le conteneur et le contenu:

In [17]:
relations = ['container', 'content', 'other relation']
extractor = RelationExtractor(model, tokenizer, relations)
cubes='the cubes'
box='the box'
ranked_rels = extractor.rank(text='the box must be filled with the cubes', head=box, tail=cubes)
print(ranked_rels)

[('container', 0.9636894315481186), ('content', 0.719708263874054), ('other relation', 0.3384249806404114)]


Le modèle est bon pour trouver les relations de contenance:

In [100]:
def contenance(text,item1,item2):
    ranked_rels = extractor.rank(text=text, head=item1, tail=item2)
    #print(ranked_rels)
    if ranked_rels[1][1]>0.5:
        return 'other relation'
    elif ranked_rels[0][1]>0.9:
        return 'container/content'
    else:
        return 'other relation'

In [106]:
text='You must put the cubes and the spheres in the boxes'
box='the boxes'
cube='the cubes'
sphere='the spheres'

In [107]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
other relation
container/content


In [108]:
text='You must put the boxes and the spheres in the cubes'
box='the boxes'
cube='the cubes'
sphere='the spheres'

In [109]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
container/content
other relation


In [104]:
text='the cubes must be filled with boxes and spheres'
box='boxes'
cube='the cubes'
sphere='spheres'

In [105]:
print(contenance(text,box,cube))
print(contenance(text,cube,sphere))
print(contenance(text,box,sphere))

container/content
container/content
other relation
